    mnist手写数字识别
    主要是为了熟悉tensorflow的运行过程

In [27]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist.input_data import read_data_sets
import math
import time

In [28]:
#载入mnist数据
input_data = read_data_sets("MNIST_DATA/", False)
batch_size = 100
image_size = 28
hidden_dim1 = 100
hidden_dim2 = 200
image_pixels = image_size * image_size
out_nums = 10
learning_rate = 0.005
max_steps = 4000

Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz


In [29]:
#构建训练模型
#初始化参数
#定义损失函数
#定义梯度下降参数
#批量训练
def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
    """Runs one evaluation against the full epoch of data.

    Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
    """
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = data_set.num_examples # FLAGS.

    num_examples = steps_per_epoch * batch_size
    for step in range(steps_per_epoch):
        images_pl, labels_pl = input_data.train.next_batch(batch_size)
        feed_dict = {images_placeholder : images_pl, labels_placeholder : labels_pl}
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = float(true_count) / num_examples
    print('Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %(num_examples, true_count, precision))

#初始化要训练的图片和标签数据的框架，
#输入数据为batch_size*image_pixels的矩阵
#输出的labels为batch_size的向量
with tf.Graph().as_default():
    images_placeholder = tf.compat.v1.placeholder(tf.float32, (batch_size, image_pixels))
    labels_placeholder = tf.compat.v1.placeholder(tf.int32, (batch_size))

    #搭建两层的卷积神经网络
    #第一层的参数设置
    with tf.compat.v1.name_scope("hidden1"):
        w1 = tf.Variable(tf.random.truncated_normal(shape=(image_pixels, hidden_dim1),
                                                    stddev=(1.0/math.sqrt(float(image_pixels)))), name="weights")
        b1 = tf.Variable(tf.zeros([hidden_dim1]), name="bias")
        hidden1 = tf.nn.relu(tf.matmul(images_placeholder, w1) + b1)
        
    with tf.compat.v1.name_scope("hidden2"):
        w2 = tf.Variable(tf.random.truncated_normal(shape=(hidden_dim1, hidden_dim2),
                                                    stddev=(1.0/math.sqrt(float(hidden_dim2)))), name="weights")
        b2 = tf.Variable(tf.zeros([hidden_dim2]), name="bias")
        hidden2 = tf.nn.relu(tf.matmul(hidden1, w2) + b2)
        
    #然后最后连接一层softmax层
    with tf.compat.v1.name_scope("softmax"):
        ws = tf.Variable(tf.random.truncated_normal(shape=[hidden_dim2, out_nums],
                                                    stddev=(1.0/math.sqrt(float(hidden_dim2)))), name="weights")
        bs = tf.Variable(tf.zeros([out_nums]), name="bias")
        logit = tf.matmul(hidden2, ws) + bs
        
    #得到卷积层的输出后计算损失函数
    loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(logits=logit, labels=tf.cast(labels_placeholder, tf.int32))
    
    #上面的这些过程可以看作是前向计算损失
    #接下来的过程就是反向传播训练参数
    tf.compat.v1.summary.scalar("loss", loss)
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate)
    global_step = tf.Variable(0, name="global_step", trainable=False)
    train_op = optimizer.minimize(loss, global_step=global_step)
    
    #得到的train_op就是训练出来的结果，然后用结果计算准确度
    correct = tf.nn.in_top_k(predictions=logit, targets=tf.cast(labels_placeholder, tf.int64), k=1)
    eval_correct = tf.reduce_sum(input_tensor=tf.cast(correct, tf.int32))
    
    summary = tf.compat.v1.summary.merge_all()
    
    #tensorflow上述只是定义了运算过程，还没有将具体的数据放在运算过程中执行，
    #接下来就是具体的运算
    #初始化所有的变量
    init = tf.compat.v1.global_variables_initializer()
    
    saver = tf.compat.v1.train.Saver()
    with tf.compat.v1.Session() as sess:
        summary_writer = tf.compat.v1.summary.FileWriter("logs/", sess.graph)
        sess.run(init)
        
        for step in range(max_steps):
            start_time = time.time()
            
            #这里是将实际的数据填入上面定义好的运算过程中
            images_feed, labels_feed = input_data.train.next_batch(batch_size)
            
            feed_dict = {images_placeholder : images_feed, labels_placeholder : labels_feed}
            
            _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)
            
            duration = time.time() - start_time
            if step % 100 == 0:
                print('Step {}: loss = {} ({} sec)'.format(step, loss_value, duration))
                
                summary_str = sess.run(summary, feed_dict=feed_dict)
                summary_writer.add_summary(summary_str, step)
                summary_writer.flush()
                
            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
                checkpoint_file = 'logs/model.ckpt'
                saver.save(sess, checkpoint_file, global_step=step)
                # Evaluate against the training set.
#                 print('Training Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        input_data.train)
                # Evaluate against the validation set.
#                 print('Validation Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        input_data.validation)
                # Evaluate against the test set.
#                 print('Test Data Eval:')
                do_eval(sess,
                        eval_correct,
                        images_placeholder,
                        labels_placeholder,
                        input_data.test)

                

Step 0: loss = 2.30354642868042 (0.2356429100036621 sec)
Step 100: loss = 2.2758095264434814 (0.002650022506713867 sec)
Step 200: loss = 2.219822883605957 (0.0027740001678466797 sec)
Step 300: loss = 2.1608426570892334 (0.002747058868408203 sec)
Step 400: loss = 2.0400402545928955 (0.003114938735961914 sec)
Step 500: loss = 1.9531081914901733 (0.0026700496673583984 sec)
Step 600: loss = 1.8012624979019165 (0.002621173858642578 sec)
Step 700: loss = 1.6865527629852295 (0.0027341842651367188 sec)
Step 800: loss = 1.387156367301941 (0.0027730464935302734 sec)
Step 900: loss = 1.257866621017456 (0.002726316452026367 sec)
Num examples: 5500000  Num correct: 4352065  Precision @ 1: 0.7913
Num examples: 500000  Num correct: 395708  Precision @ 1: 0.7914
Num examples: 1000000  Num correct: 791289  Precision @ 1: 0.7913
Step 1000: loss = 1.0945743322372437 (0.011349916458129883 sec)
Step 1100: loss = 0.8651325106620789 (0.0027430057525634766 sec)
Step 1200: loss = 0.7994837164878845 (0.00268912